In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        max_tokens=250,
        temperature=0.3,)

In [ ]:
main_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a helpful question routing assistant that routes questions provided by the user. 
                If the user greets you,you greet them back by asking their wellbeing.
                You must be able route their questions towards 2 aspects.
                If the question is about plotting graphs, charts, etc you must reply as 'IMG' only.
                If the question is regarding some other question requesting a guideline or inquiring reasons for causes regarding devices you will reply as 'RAG' only.
                If query is a greeting message you greet them back.
                If query is anything other than that you will reply as 'I don't have any information regarding this.'
                Your replies should be of single sentence and do not include 'new line' characters in your replies.""",
            ),
            ("human", "question: {question}"),
        ])

main_chain = main_prompt | llm 

In [ ]:
question = "What are the common devices failure and reasons?"

In [ ]:
ai_msg=main_chain.invoke({
        "question": question
        })
    
result = ai_msg.content
result = result.replace('\n', '')
result

In [1]:
from custom_agents.routing_agent import routing_agent_decision

In [4]:
question = "For the device Id SAQW2334 and code T between the dates 2024-08-20 and 2024-08-23, provide me a plot for the values "
result = routing_agent_decision(question)
result

'IMG'

In [1]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from utils.records import records_output
persist_directory = 'docs/chroma/'

In [2]:
def gen_context(vectordb,question):
    docs = vectordb.similarity_search(question,k=3)
    doc_list= []
    context=""
    for i in range(len(docs)):
        #print(docs[i].page_content)
        context =context + docs[i].page_content +"\n"
    for doc in docs:
        #print(doc.metadata)
        doc_list.append(doc.metadata)
    return context

In [3]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        max_tokens=250,
        temperature=0.5,)

In [4]:
main_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a helpful assistant that answers questions provided by the user. 
                If the user greets you,you greet them back by asking their wellbeing.
                Provide the answers in an explanatory manner.
                You must use the available context for providing answers. 
                Do not provide any other answers if they are not present in context.
                If you do not know about anything, then reply with I don't have enough information""",
            ),
            ("human", "context: {context}\n question: {question}"),
        ])

main_chain = main_prompt | llm 

In [6]:
question = "What are some common device failures"

In [7]:
context = gen_context(vectordb,question)

In [8]:
ai_msg=main_chain.invoke({
        "question": question,
        "context": context
        })
    
result = ai_msg.content
print(result)

I don't have enough information to answer your question. Please provide me with more context. For example, what kind of devices are you interested in? 

